# Demo 2: Handling individual polymers
This demo shows how to extract individual polymers fom chemically-complete topologies and  
assign them arbitrary metadata, including atomic partial charges, for use in polymer system preparation

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

from pathlib import Path
from polymerist.genutils.fileutils.pathutils import is_empty, assemble_path


EXAMPLE_DIR = Path('polymer_loading_examples')
assert EXAMPLE_DIR.exists() and not is_empty(EXAMPLE_DIR)

OUTPUT_DIR = Path('scratch') # dummy directory for writing without tampering with example inputs
OUTPUT_DIR.mkdir(exist_ok=True)

In [2]:
from openff.toolkit import Molecule, Topology # this import might take a sec the first time you run this cell

INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions


## Extracting individual polymer Molecules from a Topology
For convenience, we've provided a pre-made example SDF for a poly(N-isopropylacrylamide) (PNIPAAm) polymer  
NOTE: if you haven't already, please run through the [prior demo](2.1-topology_and_fragment.ipynb) to see what this file means and how to create your own

In [3]:
from polymerist.mdtools.openfftools.topology import topology_from_sdf, get_largest_offmol


pnipaam_sdf = assemble_path(EXAMPLE_DIR, 'PNIPAAm', 'sdf')
assert pnipaam_sdf.exists()

pnipaam_top = topology_from_sdf(pnipaam_sdf)
pnipaam = get_largest_offmol(pnipaam_top) # a useful trick for working with individual Molecules in a single-mol or solvated topology
pnipaam.visualize(backend='nglview')

/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/smirnoff99frosst/smirnoff99frosst.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


NGLWidget()

## Custom metadata
In principle, there's no limit to the kinds of data you can associate with a polymer, via the Molecule.properties field.  
For example, here we insert patent info about PNIPAAm which you can verify is written after the chemical table in the resulting SDF

In [4]:
from polymerist.mdtools.openfftools.topology import topology_to_sdf


pnipaam.name = 'PNIPAAm' # OpenFF Molecules also have a dedicated "name" attribute to give you molecules convenience handles
pnipaam.properties['IUPAC name'] = 'poly(N-isopropylacrylamide)'
pnipaam.properties['Common name'] = 'PNIPAAm'
pnipaam.properties['First patented'] = '1956-12-04'
pnipaam.properties['Patent holder'] = 'Edward H. Sprecht'
pnipaam.properties['Patent No.'] = 'US-2773063-A'

sdf_path_annotated = assemble_path(OUTPUT_DIR, 'PNIPAAm_annotated', 'sdf')
topology_to_sdf(sdf_path_annotated, pnipaam.to_topology()) # individual Molecule objects must be bundled into a topology before exporting to SDF

## Assigning [atomic partial charges](https://en.wikipedia.org/wiki/Partial_charge) to polymers
This is a niche-but-vital step, as it governs the electrostatic interactions within and without the polymer chains which will eventually be realized in an molecular dynamics engine  
This is a particularly difficult task for macromolecules, though speedy methods such as pretrained graph neural networks (GNNs) or [custom library charges](2.4-RCT_demo.ipynb) are available 

`polymerist` provides user-friendly hooks for various rapid partial charge wrappers (assuming you've installed one of the [parameterization toolkits](https://github.com/timbernat/polymerist?tab=readme-ov-file#2-parameterization-toolkits)).  
These hook cache the resulting partial charge values and charging method directly to the Molecule's metadata, enabling tracking of provenance during simulation preparation


In [5]:
from polymerist.mdtools.openfftools.partialcharge.molchargers import MolCharger
from polymerist.mdtools.openfftools.partialcharge.rescharge import LibraryCharger

MolCharger.subclass_registry # show which partial charge wrappers are available

{'AM1-BCC-ELF10': polymerist.mdtools.openfftools.partialcharge.molchargers.ABE10Charger,
 'Espaloma-AM1-BCC': polymerist.mdtools.openfftools.partialcharge.molchargers.EspalomaCharger,
 'NAGL': polymerist.mdtools.openfftools.partialcharge.molchargers.NAGLCharger,
 'RCT': polymerist.mdtools.openfftools.partialcharge.rescharge.interface.LibraryCharger}

In [6]:
from polymerist.mdtools.openfftools.partialcharge.molchargers import (
    NAGLCharger,  # requires having OpenFF NAGL installed
    ABE10Charger, # requires having OpenEye toolkits installed and licensed
)

# choose your partial charge method here
# charger = ABE10Charger() # NOTE: this method is unworkably slow for anything bigger than ~150 atoms; it is just placed here to indicate you COULD use AM1-BCC if desired
charger = NAGLCharger()

# assign partial charges
logging.info(f'Partial charges initial: {pnipaam.partial_charges}') # to show you nothing is up my sleeve, we verify that the charges are unset prior to the invocation

pnipaam_charged = charger.charge_molecule(pnipaam)
logging.info(f'Recorded charge method: {pnipaam_charged.properties["charge_method"]}')      # charge method is recorded for provenance
logging.info(f'Net partial charge after charger: {pnipaam_charged.partial_charges.sum()}')  # logging.info just sum to aviod cluttering output with huge array

pnipaam_charged_sdf = assemble_path(OUTPUT_DIR, 'PNIPAAm_charged', 'sdf')
topology_to_sdf(pnipaam_charged_sdf, pnipaam_charged.to_topology()) # export the charged molecule to SDF

INFO:root:Partial charges initial: None
INFO:polymerist.mdtools.openfftools.partialcharge.molchargers:Assigning partial charges via the "NAGL" method
INFO:openff.nagl.nn._models:Could not find property in lookup table: 'Could not find property value for molecule with InChI InChI=1/C162H299N27O27/c1-83(2)163-136(190)57-56-111(138(192)165-85(5)6)59-113(140(194)167-87(9)10)61-115(142(196)169-89(13)14)63-117(144(198)171-91(17)18)65-119(146(200)173-93(21)22)67-121(148(202)175-95(25)26)69-123(150(204)177-97(29)30)71-125(152(206)179-99(33)34)73-127(154(208)181-101(37)38)75-129(156(210)183-103(41)42)77-131(158(212)185-105(45)46)79-133(160(214)187-107(49)50)81-135(162(216)189-109(53)54)82-134(161(215)188-108(51)52)80-132(159(213)186-106(47)48)78-130(157(211)184-104(43)44)76-128(155(209)182-102(39)40)74-126(153(207)180-100(35)36)72-124(151(205)178-98(31)32)70-122(149(203)176-96(27)28)68-120(147(201)174-94(23)24)66-118(145(199)172-92(19)20)64-116(143(197)170-90(15)16)62-114(141(195)168-88(11)12)6